In [ ]:
#https://oranwind.org/python-pandas-ji-chu-jiao-xue-2/

import pandas as pd # 引用套件並縮寫為 pd  
from pymongo import MongoClient

def _connect_mongo(host, port, username, password, db):
    # Connect to mongo 

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)

    return conn[db]

def read_mongo(db, collection, query, host, port, username, password, no_id):
    """ Read from Mongo and Store into DataFrame """

    # Connect to mongo
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df
    
def dataframe_to_mongo(select_df, db, collection, host, port, username, password):

    # Connect to MongoDB
    database = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    collection = database[collection]   
    
    records = select_df.to_dict('records')  # 參數 record 代表把列轉成個別物件
    collection.insert_many(records)

host = 'localhost'
port = 27017
query = {}
username = None
password = None
no_id = True    
db = 'mongodb'
collection = 'collection'

# 讀取mongo
df = read_mongo(db, collection, query, host, port, username, password, no_id)

# 寫入mongo
dataframe_to_mongo(df, db, collection, host, port, username, password)
    